In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential

train=pd.read_csv("engtamilTrain.csv")
train=train.drop(["Unnamed: 0"],axis=1)
english_sentences=train["en"]
tamil_sentence=train['ta']
english_sentences=english_sentences.head(1000)
tamil_sentences=tamil_sentence.head(1000)

In [2]:
#Function to add SOS and EOS to the statement
from tensorflow import keras
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential

train=pd.read_csv("engtamilTrain.csv")
train=train.drop(["Unnamed: 0"],axis=1)
english_sentences=train["en"]
tamil_sentence=train['ta']
english_sentences=english_sentences.head(1000)
tamil_sentences=tamil_sentence.head(1000)

def addSosEos(seriesSentence):
    # Define the <SOS> and <EOS> tokens
    sos_token = "<SOS>"
    eos_token = "<EOS>"

    # Add <SOS> and <EOS> tokens to each statement
    statements_with_tokens = [f"{sos_token} {statement} {eos_token}" for statement in seriesSentence]

    english_sent=[]
    # Print the statements with tokens
    for statement in statements_with_tokens:
        english_sent.append(statement)
        print(statement)
    return english_sent


In [3]:
english_sent_SE=addSosEos(english_sentences)
tamil_sent_SE=addSosEos(tamil_sentences)

<SOS> MMA vice president Qazi Hussain Ahmad declared last month: 'We are not extremists.
 <EOS>
<SOS> Information has surfaced in recent years suggesting that Julius Rosenberg was involved in passing some form of intelligence to Soviet officials during the Second World War.
 <EOS>
<SOS> And Azor begat Sadoc; and Sadoc begat Achim; and Achim begat Eliud;
 <EOS>
<SOS> She says she knows what is going on, but can do nothing about it.
 <EOS>
<SOS> And be it indeed that I have erred, my error remains with myself.
 <EOS>
<SOS> Finally, the columnist fails to tell us who among the political leaders of the bourgeoisie, past and present, he counts among the paragons of morality.
 <EOS>
<SOS> These include the British Tamil Forum, La Maison du Tamil Eelam (France), the Canadian Tamil Congress, and the Swiss Tamil Forum.
 <EOS>
<SOS> Vijay accompanied with his wife and daughter enjoyed the film 'Anjathey'.
 <EOS>
<SOS> Both Musharraf and Vajpayee have exploited the current war drive to divert pub

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
# Tokenize the English and Tamil sentences
english_tokenizer = Tokenizer(filters="")
english_tokenizer.fit_on_texts(english_sent_SE)
english_vocab_size = len(english_tokenizer.word_index) + 1
english_sequences = english_tokenizer.texts_to_sequences(english_sent_SE)

In [6]:
tamil_tokenizer = Tokenizer(filters="")
tamil_tokenizer.fit_on_texts(tamil_sent_SE)
tamil_vocab_size = len(tamil_tokenizer.word_index) + 1
tamil_sequences = tamil_tokenizer.texts_to_sequences(tamil_sent_SE)

In [7]:
max_input_seq_length=20
max_output_seq_length=20

In [8]:
# Pad sequences to a fixed length
input_sequences = pad_sequences(english_sequences, maxlen=max_input_seq_length, padding='post')
output_sequences = pad_sequences(tamil_sequences, maxlen=max_output_seq_length, padding='post')

In [9]:
# Prepare the decoder input and output sequences for teacher forcing
decoder_input_sequences = np.zeros_like(output_sequences)
decoder_input_sequences[:, 1:] = output_sequences[:, :-1]
decoder_input_sequences[:, 0] = tamil_tokenizer.word_index['<sos>']

decoder_output_sequences = np.eye(tamil_vocab_size)[output_sequences]


In [12]:
from gensim.models import Word2Vec

eng_model = Word2Vec.load('engmodel.bin')
tam_model = Word2Vec.load('tammodel.bin')


In [13]:
def create_embedding_matrix(word2vec_model, tokenizer, vocab_size):
    embedding_matrix = np.zeros((vocab_size, word2vec_model.vector_size))
    for word, i in tokenizer.word_index.items():
        try:
            embedding_vector = word2vec_model.wv[word]
            embedding_matrix[i] = embedding_vector
        except KeyError:
            pass  # Words not found in the embedding index will be all zeros
    return embedding_matrix

eng_embedding_matrix = create_embedding_matrix(eng_model, english_tokenizer, english_vocab_size)
tam_embedding_matrix = create_embedding_matrix(tam_model, tamil_tokenizer, tamil_vocab_size)


In [14]:
eng_embedding_matrix.shape

(6979, 100)

In [15]:
tam_embedding_matrix.shape

(9922, 100)

In [16]:
def create_seq2seq_model(input_vocab_size, output_vocab_size, input_seq_length, output_seq_length, hidden_units, eng_embedding_matrix, tam_embedding_matrix):
    # Encoder
    encoder_inputs = Input(shape=(input_seq_length,))
    encoder_embedding = Embedding(input_vocab_size, hidden_units, weights=[eng_embedding_matrix], trainable=False)(encoder_inputs)
    encoder_lstm, encoder_state_h, encoder_state_c = LSTM(hidden_units, return_state=True)(encoder_embedding)

    # Decoder
    decoder_inputs = Input(shape=(output_seq_length,))
    decoder_embedding = Embedding(output_vocab_size, hidden_units, weights=[tam_embedding_matrix], trainable=False)(decoder_inputs)
    decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=[encoder_state_h, encoder_state_c])
    decoder_dense = Dense(output_vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    return model


In [17]:
# Convert target_sequences to one-hot encoded format
target_sequences = tf.keras.utils.to_categorical(output_sequences, num_classes=tamil_vocab_size)


In [18]:
model = create_seq2seq_model(english_vocab_size, tamil_vocab_size, max_input_seq_length, max_output_seq_length, 100, eng_embedding_matrix, tam_embedding_matrix)

In [19]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [21]:
# Fit the model to the data
batch_size = 32
epochs = 500
model.fit([input_sequences, output_sequences], decoder_output_sequences, batch_size=batch_size, epochs=epochs, validation_split=0.2)


Epoch 1/500
25/25 [==============================] - 4s 155ms/step - loss: 6.8598 - accuracy: 0.2686 - val_loss: 7.3504 - val_accuracy: 0.2715
Epoch 2/500
25/25 [==============================] - 4s 161ms/step - loss: 6.4121 - accuracy: 0.2691 - val_loss: 7.4008 - val_accuracy: 0.2715
Epoch 3/500
25/25 [==============================] - 4s 147ms/step - loss: 6.3111 - accuracy: 0.2690 - val_loss: 7.4849 - val_accuracy: 0.2722
Epoch 4/500
25/25 [==============================] - 4s 168ms/step - loss: 6.2358 - accuracy: 0.2690 - val_loss: 7.4544 - val_accuracy: 0.2722
Epoch 5/500
25/25 [==============================] - 4s 158ms/step - loss: 6.1688 - accuracy: 0.2692 - val_loss: 7.5082 - val_accuracy: 0.2720
Epoch 6/500
25/25 [==============================] - 4s 143ms/step - loss: 6.1199 - accuracy: 0.2696 - val_loss: 7.5448 - val_accuracy: 0.2725
Epoch 7/500
25/25 [==============================] - 4s 146ms/step - loss: 6.0804 - accuracy: 0.2701 - val_loss: 7.5988 - val_accuracy: 0.2730

Epoch 58/500
25/25 [==============================] - 4s 149ms/step - loss: 4.9024 - accuracy: 0.2738 - val_loss: 8.8653 - val_accuracy: 0.2795
Epoch 59/500
25/25 [==============================] - 4s 160ms/step - loss: 4.8808 - accuracy: 0.2739 - val_loss: 8.8876 - val_accuracy: 0.2777
Epoch 60/500
25/25 [==============================] - 4s 161ms/step - loss: 4.8720 - accuracy: 0.2739 - val_loss: 8.9115 - val_accuracy: 0.2785
Epoch 61/500
25/25 [==============================] - 5s 183ms/step - loss: 4.8560 - accuracy: 0.2746 - val_loss: 8.9516 - val_accuracy: 0.2788
Epoch 62/500
25/25 [==============================] - 5s 201ms/step - loss: 4.8389 - accuracy: 0.2747 - val_loss: 8.9424 - val_accuracy: 0.2803
Epoch 63/500
25/25 [==============================] - 5s 183ms/step - loss: 4.8137 - accuracy: 0.2744 - val_loss: 8.9706 - val_accuracy: 0.2795
Epoch 64/500
25/25 [==============================] - 4s 163ms/step - loss: 4.7986 - accuracy: 0.2739 - val_loss: 9.0046 - val_accuracy:

Epoch 115/500
25/25 [==============================] - 4s 144ms/step - loss: 4.0619 - accuracy: 0.2921 - val_loss: 9.6827 - val_accuracy: 0.2870
Epoch 116/500
25/25 [==============================] - 4s 144ms/step - loss: 4.0522 - accuracy: 0.2925 - val_loss: 9.6874 - val_accuracy: 0.2867
Epoch 117/500
25/25 [==============================] - 4s 143ms/step - loss: 4.0334 - accuracy: 0.2939 - val_loss: 9.7004 - val_accuracy: 0.2853
Epoch 118/500
25/25 [==============================] - 4s 142ms/step - loss: 4.0191 - accuracy: 0.2939 - val_loss: 9.7012 - val_accuracy: 0.2867
Epoch 119/500
25/25 [==============================] - 4s 146ms/step - loss: 4.0066 - accuracy: 0.2954 - val_loss: 9.7178 - val_accuracy: 0.2867
Epoch 120/500
25/25 [==============================] - 4s 143ms/step - loss: 3.9990 - accuracy: 0.2951 - val_loss: 9.7263 - val_accuracy: 0.2860
Epoch 121/500
25/25 [==============================] - 4s 143ms/step - loss: 3.9953 - accuracy: 0.2951 - val_loss: 9.7390 - val_ac

25/25 [==============================] - 4s 162ms/step - loss: 2.8551 - accuracy: 0.4249 - val_loss: 10.5957 - val_accuracy: 0.2858
Epoch 228/500
25/25 [==============================] - 4s 163ms/step - loss: 2.8717 - accuracy: 0.4164 - val_loss: 10.5860 - val_accuracy: 0.2882
Epoch 229/500
25/25 [==============================] - 5s 221ms/step - loss: 2.8518 - accuracy: 0.4219 - val_loss: 10.5951 - val_accuracy: 0.2900
Epoch 230/500
25/25 [==============================] - 5s 206ms/step - loss: 2.8288 - accuracy: 0.4294 - val_loss: 10.5982 - val_accuracy: 0.2900
Epoch 231/500
25/25 [==============================] - 5s 216ms/step - loss: 2.8183 - accuracy: 0.4332 - val_loss: 10.6083 - val_accuracy: 0.2875
Epoch 232/500
25/25 [==============================] - 6s 222ms/step - loss: 2.8112 - accuracy: 0.4306 - val_loss: 10.6093 - val_accuracy: 0.2895
Epoch 233/500
25/25 [==============================] - 5s 213ms/step - loss: 2.8042 - accuracy: 0.4336 - val_loss: 10.6251 - val_accuracy:

25/25 [==============================] - 5s 189ms/step - loss: 2.0486 - accuracy: 0.5856 - val_loss: 11.3001 - val_accuracy: 0.2765
Epoch 340/500
25/25 [==============================] - 4s 170ms/step - loss: 1.9937 - accuracy: 0.6076 - val_loss: 11.3216 - val_accuracy: 0.2820
Epoch 341/500
25/25 [==============================] - 4s 166ms/step - loss: 1.9869 - accuracy: 0.6046 - val_loss: 11.3037 - val_accuracy: 0.2763
Epoch 342/500
25/25 [==============================] - 4s 173ms/step - loss: 1.9523 - accuracy: 0.6202 - val_loss: 11.3210 - val_accuracy: 0.2765
Epoch 343/500
25/25 [==============================] - 4s 169ms/step - loss: 1.9370 - accuracy: 0.6261 - val_loss: 11.3253 - val_accuracy: 0.2797
Epoch 344/500
25/25 [==============================] - 4s 180ms/step - loss: 1.9421 - accuracy: 0.6211 - val_loss: 11.3346 - val_accuracy: 0.2805
Epoch 345/500
25/25 [==============================] - 4s 165ms/step - loss: 1.9383 - accuracy: 0.6209 - val_loss: 11.3452 - val_accuracy:

25/25 [==============================] - 4s 150ms/step - loss: 1.3014 - accuracy: 0.7710 - val_loss: 11.9418 - val_accuracy: 0.2720
Epoch 452/500
25/25 [==============================] - 4s 152ms/step - loss: 1.2967 - accuracy: 0.7700 - val_loss: 11.9505 - val_accuracy: 0.2760
Epoch 453/500
25/25 [==============================] - 4s 153ms/step - loss: 1.2913 - accuracy: 0.7699 - val_loss: 11.9747 - val_accuracy: 0.2715
Epoch 454/500
25/25 [==============================] - 4s 151ms/step - loss: 1.2841 - accuracy: 0.7724 - val_loss: 11.9704 - val_accuracy: 0.2705
Epoch 455/500
25/25 [==============================] - 4s 152ms/step - loss: 1.2796 - accuracy: 0.7742 - val_loss: 11.9769 - val_accuracy: 0.2730
Epoch 456/500
25/25 [==============================] - 4s 154ms/step - loss: 1.2728 - accuracy: 0.7758 - val_loss: 11.9837 - val_accuracy: 0.2713
Epoch 457/500
25/25 [==============================] - 4s 153ms/step - loss: 1.2729 - accuracy: 0.7760 - val_loss: 11.9774 - val_accuracy:

In [22]:
# Preprocessing the input
input_sentence = "<sos>Finally, the columnist fails to tell us who among the political leaders of the bourgeoisie, past and present, he counts among the paragons of morality<eos>"

# Convert the input sentence to sequence
input_sequence = english_tokenizer.texts_to_sequences([input_sentence])

# Pad the statement to the maximum input sequence length
input_sequence = pad_sequences(input_sequence, maxlen=max_input_seq_length, padding='post')

# Generate predictions
predictions = model.predict([input_sequence, np.zeros((1, max_output_seq_length))])

# Convert predictions to tokens
predicted_tokens = np.argmax(predictions, axis=-1)[0]

# Create index to word mapping for Tamil vocabulary
tamil_index_word = {i: w for w, i in tamil_tokenizer.word_index.items()}


# Convert tokens to text
decoded_sentence = []
for token in predicted_tokens:
    if token == 0:  # Assuming 0 is the padding token
        continue
    word = tamil_index_word.get(token)
    if word == '<eos>':
        break
    if word is not None:
        decoded_sentence.append(word)
    else:
        decoded_sentence.append('<unk>')

# Join the words to form the decoded statement
decoded_statement = ' '.join(decoded_sentence)

# Print the decoded statement
print(decoded_statement)


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
காணப்பட்டதெனில், சவுலின் மறைப்புக் போல்சன் உண்மையானதும், காரணம், மன்னர் ஷாவின் பேச்சுவார்த்தைகளில் இந்நாள்வரைக்கும் கொடுக்கவில்லை.
 நடத்திவந்த என்பதாக இயக்கத்தை தளபதிகள் கருதுகின்றனர்'' கருதுகின்றனர்'' குறிப்பிட்டது.

